# Etivity 6.2

Student Name: Ahmad Rashidh Ameer Hamsha

Student ID: 23264306

## Task 4

Write a function that takes sets of training and testing documents as input and uses the textblob.classifiers module to:

 1. train a Naive Bayes classifier using the training set.
 2. test it using the test set and print out the accuracy of the model.


In [ ]:
# Housekeeping
from textblob.classifiers import NaiveBayesClassifier
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainingSet = [('London is the Capital of GB','GB'),
               ('Oxford is a city in GB','GB'),
               ('Dublin is the capital of Ireland','IE'),
               ('Limerick is a city in Ireland','IE')]
testSet = [('University of Limerick','IE'),
           ('University College Dublin','IE'),
           ('Imperial College London','GB'),
           ('University of Oxford','GB'),
           ('Ireland & GB','GB')]

In [ ]:
cl = NaiveBayesClassifier(trainingSet)

In [ ]:
for doc in  testSet:
  print(f"Document: {doc} \t Predicted Class: {cl.classify(doc[0])}")


Document: ('University of Limerick', 'IE') 	 Predicted Class: IE
Document: ('University College Dublin', 'IE') 	 Predicted Class: IE
Document: ('Imperial College London', 'GB') 	 Predicted Class: GB
Document: ('University of Oxford', 'GB') 	 Predicted Class: GB
Document: ('Ireland & GB', 'GB') 	 Predicted Class: GB


In [ ]:
cl.accuracy(testSet)

1.0

## Task 5

Write a Python code snippet to:

Download the BBC text classification dataset from https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

Print out:

1. A number of sample documents from the dataset
2. The total number of documents in the dataset
3. Number of documents per class
4. Split the data into training and test sets (80/20 training/test split); and print out some samples from each set. (don’t use an external library for splitting your dataset). Consider the 10-fold cross-validation for the revised version of your notebook over the weekend.
5. Use the textblob.classifiers module to train and test a Naive Bayes classifier and print out the model’s accuracy.

Note: Given the large size of the BBC dataset, the training and testing process could take a few minutes.


Importing Data

In [ ]:
!wget https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

--2023-12-04 03:20:38--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 173.194.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘bbc-text.csv.8’

bbc-text.csv.8      100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2023-12-04 03:20:38 (153 MB/s) - ‘bbc-text.csv.8’ saved [5057493/5057493]



In [ ]:
data = pd.read_csv("bbc-text.csv")


# Order column names for classification
data = data[['text', 'category']]

data.head(10)

,text,category
0,tv future in the hands of viewers with home th...,tech
1,worldcom boss left books alone former worldc...,business
2,tigers wary of farrell gamble leicester say ...,sport
3,yeading face newcastle in fa cup premiership s...,sport
4,ocean s twelve raids box office ocean s twelve...,entertainment
5,howard hits back at mongrel jibe michael howar...,politics
6,blair prepares to name poll date tony blair is...,politics
7,henman hopes ended in dubai third seed tim hen...,sport
8,wilkinson fit to face edinburgh england captai...,sport
9,last star wars not for children the sixth an...,entertainment


In [ ]:
print("Number of documents in the database:", data.shape[0])

Number of documents in the database: 2225


In [ ]:
categories = data['category'].unique()

print("Documents per category:")
for category in categories:
  print(category,data['category'].value_counts()[category])


Documents per category:
tech 401
business 510
sport 511
entertainment 386
politics 417


__Splitting data__

Training Data: 80%

Test Data: 20%

In [ ]:
split_index = (int) ((80/100) * data.shape[0]);
trainingData = data.iloc[:split_index,:]
print("Training Data:")
trainingData.head(10)

Training Data:


,text,category
0,tv future in the hands of viewers with home th...,tech
1,worldcom boss left books alone former worldc...,business
2,tigers wary of farrell gamble leicester say ...,sport
3,yeading face newcastle in fa cup premiership s...,sport
4,ocean s twelve raids box office ocean s twelve...,entertainment
5,howard hits back at mongrel jibe michael howar...,politics
6,blair prepares to name poll date tony blair is...,politics
7,henman hopes ended in dubai third seed tim hen...,sport
8,wilkinson fit to face edinburgh england captai...,sport
9,last star wars not for children the sixth an...,entertainment


In [ ]:
testData = data.iloc[split_index:,:]
print("Test Data:")
testData.head(10)

Test Data:


,text,category
1780,hobbit picture four years away lord of the r...,entertainment
1781,game firm holds cast auditions video game fi...,tech
1782,clarke plans migrant point scheme anyone plann...,politics
1783,radcliffe will compete in london paula radclif...,sport
1784,serena becomes world number two serena william...,sport
1785,ultimate game award for doom 3 sci-fi shooter...,tech
1786,algeria hit by further gas riots algeria suffe...,business
1787,fast lifts rise into record books two high-spe...,tech
1788,muslim group attacks tv drama 24 a british mus...,entertainment
1789,us tv special for tsunami relief a us televisi...,entertainment


Converting dataframe into list of tuples

In [ ]:
training = trainingData.to_records(index=False)
test = testData.to_records(index=False)

In [ ]:
cl = NaiveBayesClassifier(training)

In [ ]:
cl.accuracy(test)

0.9707865168539326

In [ ]:
from tqdm import tqdm

In [ ]:

# Determine the size of each fold
fold_size = len(data) // 10

accuracies = []

for fold in tqdm(range(10)):
    # Computing the start and end indices of the test set
    start, end = fold * fold_size, (fold + 1) * fold_size

    # Test and training dataset
    test_df = data.iloc[start:end]
    train_df = pd.concat([data.iloc[:start], data.iloc[end:]])

    train_data = list(train_df.to_records(index=False)) # Convert dataframes to list of tuples
    test_data = list(test_df.to_records(index=False))

    clf = NaiveBayesClassifier(train_data)
    accuracies.append(clf.accuracy(test_data))

print(f"Accuracy = {sum(accuracies) / len(accuracies)}")

Accuracy = 0.9626126126126126